# Flow Strat of Cryptos

**Objective**

Attempt to predict forward returns based on trade flow. We will then set a threshold $j$ where if the predicted return is greater than the absolute value of $j$, we will consider making a trade. We will set $j$ to the 80th percentile of predicted returns during the train period. I will also analyze the betas used to make the return predictions to see if they are dependable.

For ETH/USD and BTC/USD we will look at 6, 30, 60, and 300 second trade flows and 2, 4, 6, 10, 30, and 60 second forward returns. ETH/BTC trade much less frequently, so we will look at 10, 20, 30, 60, 90, and 120 second trade flows and 10, 15, 30 second forward returns.

Once we have gathered the data for the strategies, we will look for the potential best ones. Here is the criteria:

* Beta is statistically significant
* positive correlation between predicted return and forward return
* Correlation is statistically significant
* The difference between returns after a buy signal and sell signal is statistically significant (median and mean)
* Returns after a buy signal are significantly greater than 0
* Returns after a sell signal are significantly less than 0
    
If no strategies meet the criteria, we will look at the top 3 based on correlation between predicted return and forward return

### Data Processing

In [1]:
import quandl
import pandas as pd
import numpy as np
import statsmodels.api as sm
import datetime
import functools
from scipy.stats import spearmanr
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from statsmodels.stats.stattools import durbin_watson

In [2]:
btc_usd = pd.read_table('trades_narrow_BTC-USD_2021.delim.gz', sep='\t')
eth_btc = pd.read_table('trades_narrow_ETH-BTC_2021.delim.gz', sep='\t')
eth_usd = pd.read_table('trades_narrow_ETH-USD_2021.delim.gz', sep='\t')

In [3]:
eth_btc.head()

,received_utc_nanoseconds,timestamp_utc_nanoseconds,PriceMillionths,SizeBillionths,Side
0,1618090137140737000,1618090137157544000,35690,1000000,-1
1,1618090137851379000,1618090137864544000,35700,29801980,2
2,1618090143777062000,1618090143793909000,35700,4641070,1
3,1618090155038792000,1618090155053606000,35710,1000000,1
4,1618090155039208000,1618090155053606000,35720,231355690,1


In [4]:
# convert dates to timestamps
btc_usd['received_utc_nanoseconds'] = btc_usd['received_utc_nanoseconds'].apply(pd.Timestamp)
btc_usd['timestamp_utc_nanoseconds'] = btc_usd['timestamp_utc_nanoseconds'].apply(pd.Timestamp)

# set the received timestamp as the index
btc_usd = btc_usd.set_index('received_utc_nanoseconds')
btc_usd['side_size'] = btc_usd['Side']*btc_usd['SizeBillionths']
#--------------------------------------------------------

# convert dates to timestamps
eth_usd['received_utc_nanoseconds'] = eth_usd['received_utc_nanoseconds'].apply(pd.Timestamp)
eth_usd['timestamp_utc_nanoseconds'] = eth_usd['timestamp_utc_nanoseconds'].apply(pd.Timestamp)

# set the received timestamp as the index
eth_usd = eth_usd.set_index('received_utc_nanoseconds')
eth_usd['side_size'] = eth_usd['Side']*eth_usd['SizeBillionths']

#--------------------------------------------------------

# convert dates to timestamps
eth_btc['received_utc_nanoseconds'] = eth_btc['received_utc_nanoseconds'].apply(pd.Timestamp)
eth_btc['timestamp_utc_nanoseconds'] = eth_btc['timestamp_utc_nanoseconds'].apply(pd.Timestamp)

# set the received timestamp as the index
eth_btc = eth_btc.set_index('received_utc_nanoseconds')
eth_btc['side_size'] = eth_btc['Side']*eth_btc['SizeBillionths']

In [5]:
def tau_trade_flow(cummulative_trades,Tau):
    """Calculate trade flow from cummulative trade sides over Tau time period."""
    
    cummulative_trades = cummulative_trades.sort_index()
    
    T_cummulative_trades = cummulative_trades.reindex(cummulative_trades.index-pd.Timedelta(Tau), method='bfill')
    T_cummulative_trades.index = cummulative_trades.index

    trade_flow = cummulative_trades-T_cummulative_trades
    trade_flow_i = trade_flow.shift(1)
    return trade_flow_i

def T_fwd_rtn(price_series,T):
    """Calculate T forward returns"""
    
    price_series = price_series.sort_index()

    T_fwd_prices = price_series.reindex(price_series.index+pd.Timedelta(T), method='ffill')
    T_fwd_prices.index = price_series.index
    T_fwd_rtns = T_fwd_prices/price_series-1
    
    T_fwd_rtns.name = 'fwd_rtn_'+T

    return T_fwd_rtns

In [6]:
time_diffs = []
for i in range(0,round(len(eth_btc)*.2)-1):
    time_diff = eth_btc.index[i+1]-eth_btc.index[i]
    time_diffs.append(time_diff)
    
print('average time between ETH/BTC trades:{} seconds'.format(np.mean(time_diffs).seconds))
print('median time between ETH/BTC trades:{} seconds'.format(np.median(time_diffs).seconds))

average time between ETH/BTC trades:6 seconds
median time between ETH/BTC trades:4 seconds


In [7]:
eth_btc_trade_flows = pd.DataFrame({'flow_10s':tau_trade_flow(eth_btc['side_size'].cumsum(),'10s'),
                                    'flow_20s':tau_trade_flow(eth_btc['side_size'].cumsum(),'20s'),
                                    'flow_30s':tau_trade_flow(eth_btc['side_size'].cumsum(),'30s'),
                                    'flow_60s':tau_trade_flow(eth_btc['side_size'].cumsum(),'60s'),
                                    'flow_90s':tau_trade_flow(eth_btc['side_size'].cumsum(),'90s'),
                                    'flow_120s':tau_trade_flow(eth_btc['side_size'].cumsum(),'120s')})

eth_btc_fwd_rtns = pd.DataFrame({'fwd_rtn_10s':T_fwd_rtn(eth_btc['PriceMillionths'],'10s'),
                                 'fwd_rtn_15s':T_fwd_rtn(eth_btc['PriceMillionths'],'15s'),
                                 'fwd_rtn_30s':T_fwd_rtn(eth_btc['PriceMillionths'],'30s')})

In [8]:
time_diffs = []
for i in range(0,round(len(btc_usd)*.2)-1):
    time_diff = btc_usd.index[i+1]-btc_usd.index[i]
    time_diffs.append(time_diff)
    
print('average time between BTC/USD trades:{} seconds'.format(np.mean(time_diffs).seconds))
print('median time between BTC/USD trades:{} seconds'.format(np.median(time_diffs).seconds))

average time between BTC/USD trades:0 seconds
median time between BTC/USD trades:0 seconds


In [9]:
btc_usd_trade_flows = pd.DataFrame({'flow_6s':tau_trade_flow(btc_usd['side_size'].cumsum(),'6s'),
                                   'flow_30s':tau_trade_flow(btc_usd['side_size'].cumsum(),'30s'),
                                   'flow_60s':tau_trade_flow(btc_usd['side_size'].cumsum(),'60s'),
                                   'flow_300s':tau_trade_flow(btc_usd['side_size'].cumsum(),'300s')})

btc_usd_fwd_rtns = pd.DataFrame({'fwd_rtn_2s':T_fwd_rtn(btc_usd['PriceMillionths'],'2s'),
                                 'fwd_rtn_4s':T_fwd_rtn(btc_usd['PriceMillionths'],'4s'),
                                 'fwd_rtn_6s':T_fwd_rtn(btc_usd['PriceMillionths'],'6s'),
                                 'fwd_rtn_10s':T_fwd_rtn(btc_usd['PriceMillionths'],'10s'),
                                 'fwd_rtn_30s':T_fwd_rtn(btc_usd['PriceMillionths'],'30s'),
                                 'fwd_rtn_60s':T_fwd_rtn(btc_usd['PriceMillionths'],'60s')})

In [10]:
time_diffs = []
for i in range(0,round(len(eth_usd)*.2)-1):
    time_diff = eth_usd.index[i+1]-eth_usd.index[i]
    time_diffs.append(time_diff)
    
print('average time between ETH/USD trades:{} seconds'.format(np.mean(time_diffs).seconds))
print('median time between ETH/USD trades:{} seconds'.format(np.median(time_diffs).seconds))

average time between ETH/USD trades:0 seconds
median time between ETH/USD trades:0 seconds


In [11]:
eth_usd_trade_flows = pd.DataFrame({'flow_6s':tau_trade_flow(eth_usd['side_size'].cumsum(),'6s'),
                                   'flow_30s':tau_trade_flow(eth_usd['side_size'].cumsum(),'30s'),
                                   'flow_60s':tau_trade_flow(eth_usd['side_size'].cumsum(),'60s'),
                                   'flow_300s':tau_trade_flow(eth_usd['side_size'].cumsum(),'300s')})

eth_usd_fwd_rtns = pd.DataFrame({'fwd_rtn_2s':T_fwd_rtn(eth_usd['PriceMillionths'],'2s'),
                                 'fwd_rtn_4s':T_fwd_rtn(eth_usd['PriceMillionths'],'4s'),
                                 'fwd_rtn_6s':T_fwd_rtn(eth_usd['PriceMillionths'],'6s'),
                                 'fwd_rtn_10s':T_fwd_rtn(eth_usd['PriceMillionths'],'10s'),
                                 'fwd_rtn_30s':T_fwd_rtn(eth_usd['PriceMillionths'],'30s'),
                                 'fwd_rtn_60s':T_fwd_rtn(eth_usd['PriceMillionths'],'60s')})


In [12]:
btc_usd_flows_rtns = btc_usd_trade_flows.join(btc_usd_fwd_rtns)
btc_usd_flows_rtns = btc_usd_flows_rtns.drop(index=btc_usd_flows_rtns.index[0], axis=0)
                                                                        
btc_usd_train_cutoff = round(len(btc_usd_flows_rtns)*.2)

btc_usd_train = btc_usd_flows_rtns.iloc[:btc_usd_train_cutoff,:]
btc_usd_test = btc_usd_flows_rtns.iloc[btc_usd_train_cutoff:,:]                                                                        
                             

In [13]:
eth_btc_flows_rtns = eth_btc_trade_flows.join(eth_btc_fwd_rtns)
eth_btc_flows_rtns = eth_btc_flows_rtns.drop(index=eth_btc_flows_rtns.index[0], axis=0)

eth_btc_train_cutoff = round(len(eth_btc_flows_rtns)*.2)

eth_btc_train = eth_btc_flows_rtns.iloc[:eth_btc_train_cutoff,:]
eth_btc_test = eth_btc_flows_rtns.iloc[eth_btc_train_cutoff:,:]

In [14]:
eth_usd_flows_rtns = eth_usd_trade_flows.join(eth_usd_fwd_rtns)
eth_usd_flows_rtns = eth_usd_flows_rtns.drop(index=eth_usd_flows_rtns.index[0], axis=0)

eth_usd_train_cutoff = round(len(eth_usd_flows_rtns)*.2)

eth_usd_train = eth_usd_flows_rtns.iloc[:eth_usd_train_cutoff,:]
eth_usd_test = eth_usd_flows_rtns.iloc[eth_usd_train_cutoff:,:]

In [15]:
def find_beta(df, flow_period, rtn_period):
    """Find the beta of the trade flow to the forward return."""
    
    rtn_mod = sm.OLS(endog = df[rtn_period], exog = df[flow_period])
    mod_res = rtn_mod.fit()
    mod_beta = mod_res.params[0]
    beta_pval = mod_res.pvalues[0]
    
    return (mod_beta,beta_pval)

def predict_fwd_rtns(df_train,df_test,flow_period, rtn_period):
    """Calculate the beta of forward returns and predict returns on the test set"""
    
    
    fwd_rtn_beta = find_beta(df_train,flow_period, rtn_period)
    
    predict_df = df_test.copy()
    
    predict_df['predicted_rtn'] = predict_df[flow_period]*fwd_rtn_beta[0]
    
    fwd_rtn_df = predict_df.loc[:,[flow_period, rtn_period, 'predicted_rtn']]
    
    return (fwd_rtn_beta,fwd_rtn_df)
    

In [16]:
def test_strategies(df_train, df_test,strategies):
    
    betas = []
    beta_pvals = []
    flow_periods = []
    rtn_periods = []
    cors = []
    cor_p_values = []
    buy_signal_counts = []
    sell_signal_counts = []
    buy_sell_med_results = []
    buy_sell_mean_results = []
    buy_sig_test_results = []
    sell_sig_test_results = []
    
    k = 0

    return_dict = {}
    
    for strategy in strategies:     
        
        predicted_values = predict_fwd_rtns(df_train=df_train,
                     df_test=df_test,
                     flow_period=strategy[0],
                     rtn_period=strategy[1])
        
        predicted_beta = predicted_values[0][0]
        predicted_beta_pval = predicted_values[0][1]
        
        predicted_df = predicted_values[1]
        
        df_train_predicted_rtns = df_train.loc[df_train[strategy[1]]>0,strategy[1]]*predicted_beta
        
        j = df_train_predicted_rtns.quantile(0.8)
        
        predicted_df['buy_signal'] = predicted_df['predicted_rtn'] > j
        predicted_df['sell_signal'] = predicted_df['predicted_rtn'] < -j
        
        buy_signal_rtns = predicted_df.loc[predicted_df['buy_signal'],strategy[1]]
        sell_signal_rtns = predicted_df.loc[predicted_df['sell_signal'],strategy[1]]

        median_diff_test = stats.kruskal(buy_signal_rtns,sell_signal_rtns)[1]
        mean_diff_test = stats.f_oneway(buy_signal_rtns,sell_signal_rtns)[1]

        buys_sig_test = stats.ttest_1samp(buy_signal_rtns,0,alternative='greater')[1]
        sells_sig_test = stats.ttest_1samp(sell_signal_rtns,0,alternative='less')[1]
        
        return_dict[strategy] = predicted_df
        
        buy_signal_count = sum(predicted_df['buy_signal'])
        sell_signal_count = sum(predicted_df['sell_signal'])
        
        corr_pred_rtn = spearmanr(predicted_df[strategy[1]],predicted_df['predicted_rtn'])

        betas.append(predicted_beta)
        beta_pvals.append(predicted_beta_pval)
        flow_periods.append(strategy[0])
        rtn_periods.append(strategy[1])
        cors.append(corr_pred_rtn[0])
        cor_p_values.append(round(corr_pred_rtn[1],4))
        buy_signal_counts.append(buy_signal_count)
        sell_signal_counts.append(sell_signal_count)
        buy_sell_med_results.append(median_diff_test)
        buy_sell_mean_results.append(mean_diff_test)
        buy_sig_test_results.append(buys_sig_test)
        sell_sig_test_results.append(sells_sig_test)

    results_df = pd.DataFrame({'Flow Period':flow_periods,
                               'Return Period':rtn_periods,
                               'Beta':betas,
                               'Beta p-value':beta_pvals,
                               'Correlation':cors,
                               'Correlation p-value':cor_p_values,
                              'Buy Signal Count':buy_signal_counts,
                              'Sell Signal Count':sell_signal_counts,
                              'Buy Sell Median Difference Test':buy_sell_med_results,
                              'Buy Sell Mean Difference Test':buy_sell_mean_results,
                              'Buy Signal Return Significance':buy_sig_test_results,
                              'Sell Signal Return Significance':sell_sig_test_results})
    
    results_df = results_df.sort_values('Correlation',ascending=False)
    
    k = k+1
    
    return (results_df,return_dict)


In [17]:
btc_usd_params = [('flow_6s','fwd_rtn_2s'),
                          ('flow_6s','fwd_rtn_4s'),
                          ('flow_6s','fwd_rtn_6s'),
                          ('flow_30s','fwd_rtn_2s'),
                          ('flow_30s','fwd_rtn_4s'),
                          ('flow_30s','fwd_rtn_6s'),
                          ('flow_30s','fwd_rtn_10s'),
                          ('flow_60s','fwd_rtn_2s'),
                          ('flow_60s','fwd_rtn_4s'),
                          ('flow_60s','fwd_rtn_6s'),
                          ('flow_60s','fwd_rtn_10s'),
                          ('flow_300s','fwd_rtn_6s'),
                          ('flow_300s','fwd_rtn_10s'),
                          ('flow_300s','fwd_rtn_30s'),
                          ('flow_300s','fwd_rtn_60s')]

eth_usd_params = [('flow_6s','fwd_rtn_2s'),
                          ('flow_6s','fwd_rtn_4s'),
                          ('flow_6s','fwd_rtn_6s'),
                          ('flow_30s','fwd_rtn_2s'),
                          ('flow_30s','fwd_rtn_4s'),
                          ('flow_30s','fwd_rtn_6s'),
                          ('flow_30s','fwd_rtn_10s'),
                          ('flow_60s','fwd_rtn_2s'),
                          ('flow_60s','fwd_rtn_4s'),
                          ('flow_60s','fwd_rtn_6s'),
                          ('flow_60s','fwd_rtn_10s'),
                          ('flow_300s','fwd_rtn_6s'),
                          ('flow_300s','fwd_rtn_10s'),
                          ('flow_300s','fwd_rtn_30s'),
                          ('flow_300s','fwd_rtn_60s')]

eth_btc_params = [('flow_10s','fwd_rtn_10s'),
                  ('flow_20s','fwd_rtn_10s'),
                  ('flow_20s','fwd_rtn_15s'),
                  ('flow_30s','fwd_rtn_10s'),
                  ('flow_30s','fwd_rtn_15s'),
                  ('flow_30s','fwd_rtn_30s'),
                  ('flow_60s','fwd_rtn_10s'),
                  ('flow_60s','fwd_rtn_15s'),
                  ('flow_60s','fwd_rtn_30s'),
                  ('flow_90s','fwd_rtn_10s'),
                  ('flow_90s','fwd_rtn_15s'),
                  ('flow_90s','fwd_rtn_30s'),
                 ('flow_120s','fwd_rtn_10s'),
                 ('flow_120s','fwd_rtn_15s'),
                 ('flow_120s','fwd_rtn_30s')]

In [18]:
eth_btc_strategies = test_strategies(df_train=eth_btc_train,df_test=eth_btc_test,strategies=eth_btc_params)
eth_usd_strategies = test_strategies(df_train=eth_usd_train, df_test=eth_usd_test,strategies=eth_usd_params)
btc_usd_strategies = test_strategies(df_train=btc_usd_train, df_test=btc_usd_test,strategies=btc_usd_params)

In [19]:
btc_usd_results = btc_usd_strategies[0]

btc_usd_criteria = (btc_usd_results['Beta p-value'] < 0.5)&\
(btc_usd_results['Correlation p-value']<0.05)&\
(btc_usd_results['Buy Sell Median Difference Test']<0.05)&\
(btc_usd_results['Buy Sell Mean Difference Test']<0.05)&\
(btc_usd_results['Buy Signal Return Significance']<0.05)&\
(btc_usd_results['Sell Signal Return Significance']<0.05)

if(len(btc_usd_results.loc[btc_usd_criteria,:])==0):
    btc_usd_strats_to_analyze = btc_usd_results.head(3)
else:
    btc_usd_strats_to_analyze = btc_usd_results.loc[btc_usd_criteria,:]

In [20]:
eth_usd_results = eth_usd_strategies[0]


eth_usd_results = eth_usd_strategies[0]

eth_usd_criteria = (eth_usd_results['Beta p-value'] < 0.5)&\
(eth_usd_results['Correlation p-value']<0.05)&\
(eth_usd_results['Buy Sell Median Difference Test']<0.05)&\
(eth_usd_results['Buy Sell Mean Difference Test']<0.05)&\
(eth_usd_results['Buy Signal Return Significance']<0.05)&\
(eth_usd_results['Sell Signal Return Significance']<0.05)



if(len(eth_usd_results.loc[eth_usd_criteria,:])==0):
    eth_usd_strats_to_analyze = eth_usd_results.head(3).reset_index(drop=True)
else:
    eth_usd_strats_to_analyze = eth_usd_results.loc[eth_usd_criteria,:].reset_index(drop=True)

In [21]:
eth_btc_strategies[0]

eth_btc_results = eth_btc_strategies[0]

eth_btc_criteria = (eth_btc_results['Beta p-value'] < 0.5)&\
(eth_btc_results['Correlation p-value']<0.05)&\
(eth_btc_results['Buy Sell Median Difference Test']<0.05)&\
(eth_btc_results['Buy Sell Mean Difference Test']<0.05)&\
(eth_btc_results['Buy Signal Return Significance']<0.05)&\
(eth_btc_results['Sell Signal Return Significance']<0.05)


eth_btc_strats_to_analyze = eth_btc_results.loc[eth_btc_criteria,:].reset_index(drop=True)

if(len(eth_btc_results.loc[eth_btc_criteria,:])==0):
    eth_btc_strats_to_analyze = eth_btc_results.head(3)
else:
    eth_btc_strats_to_analyze = eth_btc_results.loc[eth_btc_criteria,:].reset_index(drop=True)

In [22]:
eth_btc_strats = []

for i in range(0,len(eth_btc_strats_to_analyze)):
    eth_btc_strats.append((eth_btc_strats_to_analyze['Flow Period'].iloc[i],
                           eth_btc_strats_to_analyze['Return Period'].iloc[i]))

flow_periods = []
return_periods = []
kstest_pvalues = []
dw_tstats = []

for strat in eth_btc_strats:
    strat_rtns = eth_btc_strategies[1][strat]

    strat_residuals = strat_rtns['predicted_rtn']-strat_rtns[strat[1]]
    ks_test_pvalue = stats.kstest(strat_residuals,'norm')[0]
    durbin_watson_tstat = durbin_watson(strat_residuals)
    
    flow_periods.append(strat[0])
    return_periods.append(strat[1])
    kstest_pvalues.append(ks_test_pvalue)
    dw_tstats.append(durbin_watson_tstat)
    
assumption_dat = pd.DataFrame({'Flow Period':flow_periods,
                               'Return Period':return_periods,
                               'Normality Test p-value':kstest_pvalues,
                               'Autocorrelation t-stat':dw_tstats})

eth_btc_strategy_table = eth_btc_strats_to_analyze.merge(assumption_dat, how='left', on=['Flow Period','Return Period'])

In [23]:
btc_usd_strats = []

for i in range(0,len(btc_usd_strats_to_analyze)):
    btc_usd_strats.append((btc_usd_strats_to_analyze['Flow Period'].iloc[i],
                           btc_usd_strats_to_analyze['Return Period'].iloc[i]))

flow_periods = []
return_periods = []
kstest_pvalues = []
dw_tstats = []

for strat in btc_usd_strats:
    strat_rtns = btc_usd_strategies[1][strat]

    strat_residuals = strat_rtns['predicted_rtn']-strat_rtns[strat[1]]
    ks_test_pvalue = stats.kstest(strat_residuals,'norm')[0]
    durbin_watson_tstat = durbin_watson(strat_residuals)
    
    flow_periods.append(strat[0])
    return_periods.append(strat[1])
    kstest_pvalues.append(ks_test_pvalue)
    dw_tstats.append(durbin_watson_tstat)
    
assumption_dat = pd.DataFrame({'Flow Period':flow_periods,
                               'Return Period':return_periods,
                               'Normality Test p-value':kstest_pvalues,
                               'Autocorrelation t-stat':dw_tstats})

btc_usd_strategy_table = btc_usd_strats_to_analyze.merge(assumption_dat, how='left', on=['Flow Period','Return Period'])

In [24]:
eth_usd_strats = []

for i in range(0,len(eth_usd_strats_to_analyze)):
    eth_usd_strats.append((eth_usd_strats_to_analyze['Flow Period'].iloc[i],
                           eth_usd_strats_to_analyze['Return Period'].iloc[i]))

flow_periods = []
return_periods = []
kstest_pvalues = []
dw_tstats = []

for strat in eth_usd_strats:
    strat_rtns = eth_usd_strategies[1][strat]

    strat_residuals = strat_rtns['predicted_rtn']-strat_rtns[strat[1]]
    ks_test_pvalue = stats.kstest(strat_residuals,'norm')[0]
    durbin_watson_tstat = durbin_watson(strat_residuals)
    
    flow_periods.append(strat[0])
    return_periods.append(strat[1])
    kstest_pvalues.append(ks_test_pvalue)
    dw_tstats.append(durbin_watson_tstat)
    
assumption_dat = pd.DataFrame({'Flow Period':flow_periods,
                               'Return Period':return_periods,
                               'Normality Test p-value':kstest_pvalues,
                               'Autocorrelation t-stat':dw_tstats})

eth_usd_strategy_table = eth_usd_strats_to_analyze.merge(assumption_dat, how='left', on=['Flow Period','Return Period'])

In [25]:
numeric_columns = ['Beta', 'Beta p-value', 'Correlation','Correlation p-value', 'Buy Sell Median Difference Test', 'Buy Sell Mean Difference Test',
       'Buy Signal Return Significance', 'Sell Signal Return Significance',
       'Normality Test p-value', 'Autocorrelation t-stat']


### Results

In [26]:
eth_btc_strategy_table.loc[:,numeric_columns] = round(eth_btc_strategy_table.loc[:,numeric_columns],4)

eth_btc_strategy_table

,Flow Period,Return Period,Beta,Beta p-value,Correlation,Correlation p-value,Buy Signal Count,Sell Signal Count,Buy Sell Median Difference Test,Buy Sell Mean Difference Test,Buy Signal Return Significance,Sell Signal Return Significance,Normality Test p-value,Autocorrelation t-stat
0,flow_30s,fwd_rtn_30s,-0.0,0.0112,0.0303,0.0000,10861,15520,0.0000,0.0062,0.1506,0.0010,0.4990,0.6168
1,flow_60s,fwd_rtn_30s,-0.0,0.6469,0.0278,0.0000,10470,15725,0.0000,0.0023,0.1126,0.0004,0.4990,0.6178
2,flow_60s,fwd_rtn_15s,-0.0,0.3114,0.0218,0.0004,10470,15725,0.0001,0.0020,0.2333,0.0000,0.4992,0.9346


**ETH/BTC Strategies Summary**

None of the strategies for trading ethereum in bitcoin met the criteria to be considered potentially profitable. Looking at the top 3 strategies by correlation of predicted return and forward return, the 30 second return period seems to have the most potential. Some of the other strategies that appear to work for ETH/USD and BTC/USD aren't feasible with ETH/BTC because it doesn't trade frequently enough. Additionally, none of the betas for any of the strategies that were tested appear to be significant except 30 second flow and 30 second forward return.

In [27]:
eth_usd_strategy_table.loc[:,numeric_columns] = round(eth_usd_strategy_table.loc[:,numeric_columns],4)

eth_usd_strategy_table

,Flow Period,Return Period,Beta,Beta p-value,Correlation,Correlation p-value,Buy Signal Count,Sell Signal Count,Buy Sell Median Difference Test,Buy Sell Mean Difference Test,Buy Signal Return Significance,Sell Signal Return Significance,Normality Test p-value,Autocorrelation t-stat
0,flow_6s,fwd_rtn_2s,0.0,0.0000,0.0716,0.0,160409,84666,0.0,0.0,0.0,0.0,0.4989,0.3139
1,flow_6s,fwd_rtn_6s,0.0,0.0000,0.0700,0.0,160409,84666,0.0,0.0,0.0,0.0,0.4987,0.1499
2,flow_6s,fwd_rtn_4s,0.0,0.0000,0.0688,0.0,160409,84666,0.0,0.0,0.0,0.0,0.4989,0.2067
3,flow_30s,fwd_rtn_2s,0.0,0.0000,0.0560,0.0,159744,85694,0.0,0.0,0.0,0.0,0.4991,0.3346
4,flow_30s,fwd_rtn_6s,0.0,0.0000,0.0515,0.0,159744,85694,0.0,0.0,0.0,0.0,0.4987,0.1500
5,flow_30s,fwd_rtn_4s,0.0,0.0000,0.0514,0.0,159744,85694,0.0,0.0,0.0,0.0,0.4989,0.2093
6,flow_30s,fwd_rtn_10s,0.0,0.0000,0.0465,0.0,159744,85694,0.0,0.0,0.0,0.0,0.4984,0.0935
7,flow_60s,fwd_rtn_2s,0.0,0.0000,0.0452,0.0,159367,86071,0.0,0.0,0.0,0.0,0.4991,0.3384
8,flow_60s,fwd_rtn_4s,0.0,0.0000,0.0406,0.0,159367,86071,0.0,0.0,0.0,0.0,0.4989,0.2105
9,flow_60s,fwd_rtn_6s,0.0,0.0282,0.0389,0.0,159367,86071,0.0,0.0,0.0,0.0,0.4987,0.1503


**ETH/USD Strategies Summary**

Trading Ethereum in USD has the most strategies that are potentially profitable and matched our criteria. There is a decent range of flow periods and return periods that appear to be profitable. The strategies generate plenty of signals and are showing forward returns greater than 0 when there is a buy signal and less than 0 when there is a sell signal. The betas for the strategy appear to have normally distributed residuals.

In [28]:
btc_usd_strategy_table.loc[:,numeric_columns] = round(btc_usd_strategy_table.loc[:,numeric_columns],4)

btc_usd_strategy_table

,Flow Period,Return Period,Beta,Beta p-value,Correlation,Correlation p-value,Buy Signal Count,Sell Signal Count,Buy Sell Median Difference Test,Buy Sell Mean Difference Test,Buy Signal Return Significance,Sell Signal Return Significance,Normality Test p-value,Autocorrelation t-stat
0,flow_6s,fwd_rtn_4s,0.0,0.0,0.0938,0.0,2898439,2116181,0.0,0.0,0.0,0.0,0.4974,0.0612
1,flow_6s,fwd_rtn_6s,0.0,0.0,0.0880,0.0,2898439,2116181,0.0,0.0,0.0,0.0,0.4971,0.0466
2,flow_30s,fwd_rtn_10s,0.0,0.0,0.0445,0.0,2731043,2284026,0.0,0.0,0.0,0.0,0.4965,0.0298


**BTC/USD Strategies Summary**

Trading bitcoin in USD also has several strategies that are potentially profitable and matched our criteria. Using 6 second flow to predict 6 and 4 second forward returns are especially interesting. The strategies generate plenty of signals and are showing forward returns greater than 0 when there is a buy signal and less than 0 when there is a sell signal. Having the longer time frames will also potentially make them easier to implement because latency won't be as big of an issue. The betas for the strategy appear to have normally distributed residuals but look to potentially have autocorrelation. More work would need to be done before implementing these strategies to make the betas more robust.